In [ ]:
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import os
from img2vec_pytorch import Img2Vec
from PIL import Image
import xml.etree.ElementTree as ET
from urllib.parse import urlparse, unquote

: 

In [58]:
elastic_client = Elasticsearch(hosts=['http://localhost:9200'],
                               basic_auth=('elastic', 'master'))
model = Img2Vec()

In [59]:
def generate_embedding(filename):
    img = Image.open(filename).convert("RGB")
    vec = model.get_vec(img)
    return vec.tolist()

In [60]:
def build_doc(no:str, name:str, class_label, class_id, vector):
  document={ 
    "no": no,
    "name": name,
    "class_label": class_label, 
    "class_id": class_id, 
    "embedding": vector
  }   
  return document

In [61]:
#def get_files(path:str, extension:str):
#    file_list = [f for f in os.listdir(path) if f.endswith(extension)]
#    return file_list

In [62]:
#def get_classes(path:str, file_name:str):
#    tree = ET.parse(path+file_name)
#    root = tree.getroot()
#    classes=[]
#    for child in root:
#        if (child.tag == 'object'):
#            for new_child in child:
#                if (new_child.tag == 'name'):
#                    classes.append(new_child.text)
#    return classes

In [63]:
# dataset_path = "datasets/small/images/"
# dataset_file = "datasets/small/sb_training.csv"
# index_name = "bs_index"
dataset_path = "datasets/full/images/"
dataset_file = "datasets/full/sb_training.csv"
index_name = "bs_index_full"

In [64]:
low_memory=False
df = pd.read_csv(dataset_file)
df

,foto,class_id,padrao_construtivo
0,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio
1,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio
2,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio
3,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo
4,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo
...,...,...,...
1131,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo
1132,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo
1133,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo
1134,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio


In [65]:
def extract_filename(url):
    path = urlparse(url).path
    # Desconsiderando barras no final do caminho e decodificando caracteres especiais
    return unquote(path.split("/")[-1])

In [66]:
df['arquivo'] = df['foto'].apply(extract_filename)
df

,foto,class_id,padrao_construtivo,arquivo
0,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio,33.1_1570625004221.jpg
1,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio,05_1599845566225.jpg
2,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio,17.1_1569948160184.jpg
3,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo,1_1569339287149.jpg
4,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo,04_1596823991068.jpg
...,...,...,...,...
1131,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo,6_1579116191029.jpg
1132,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo,30_1572866696625.jfif
1133,https://geo-criciuma-fotos.s3-us-east-2.amazon...,4,Médio Baixo,01_1570621654126.jpg
1134,https://geo-criciuma-fotos.s3-us-east-2.amazon...,3,Médio,66_1570042165580.jpg


In [67]:
id = 0
actions = []
for index, row in df.iterrows():
    id += 1
    image_file = dataset_path+row.arquivo
    class_label = row.padrao_construtivo
    class_id = row.class_id
    # print(image_file)
    vector = generate_embedding(image_file)
    actions.append({"_index":index_name.lower(), 
                    "_source": build_doc(id, image_file, class_label, class_id, vector)})
    if ((id % 50 ) == 0):
      bulk(elastic_client, actions)
      actions.clear()
      print("{} images were indexed!!!".format(id))

if len(actions) != 0:
  bulk(elastic_client, actions)
  actions.clear()
  print("{} images were indexed!!!".format(id))
print("Indexing was finished!!!")


50 images were indexed!!!
100 images were indexed!!!
150 images were indexed!!!
200 images were indexed!!!
250 images were indexed!!!
300 images were indexed!!!
350 images were indexed!!!
400 images were indexed!!!
450 images were indexed!!!
500 images were indexed!!!
550 images were indexed!!!
600 images were indexed!!!
650 images were indexed!!!
700 images were indexed!!!
750 images were indexed!!!
800 images were indexed!!!
850 images were indexed!!!
900 images were indexed!!!
950 images were indexed!!!
1000 images were indexed!!!
1050 images were indexed!!!
1100 images were indexed!!!
1136 images were indexed!!!
Indexing was finished!!!
